# Future Talk


## Monitoring and Debugging Deep Neural Networks



by Adrian Wälchli


In [1]:
import torch
torch.__version__

'1.1.0'

Code: [github.com/awaelchli/pytorch-demo](https://github.com/awaelchli/pytorch-demo)

## Topics

- Introduction to Weights and Biases
    - Error Curves
    - Images
    - Histograms
    - Hyperparameter Tracking



- Best Practices
    - Managing Runs
    - Datasets
    - Checkpointing and Resuming
    - Multi-GPU
    - Miscellaneous Tipps and Tricks

- Debugging Neural Networks
    - Input and Output
    - Batch Normalization
    - Dropout
    - Classification Caveats
    - Gradients
    - Adversarial Examples

### Let's start easy ...

with the good old [MNIST](./01_MNIST_Basic.ipynb) example!

### What It Should Look Like

#### Logging metrics in different runs
<img src="./figures/logging2.png" width="80%"/>

#### Plotting error curves
<img src="./figures/logging1.png" width="50%"/>

#### Visualizing and exploring data
<img src="./figures/logging3.png" width="50%"/>

### Weights & Biases
##### Double You and Bee
1. Sign up with GitHub or Google Account at https://www.wandb.com
2. Copy the API key
3. Log in
```
conda activate pytorch-demo
wandb login API_KEY
```
4. Create a project


See also: https://docs.wandb.com/

### Usage

Call the ```init``` function once to setup your run.

In [2]:
import wandb

wandb.init(
    name='Introduction to W & B', 
    config=dict(),  
    project='pytorch-demo', 
    tags=['baseline'],
    dir='./runs',
    entity='awaelchli',
    group='slides',
    resume=False,
)

W&B Run: https://app.wandb.ai/awaelchli/pytorch-demo/runs/0vv7ev71
Call `%%wandb` in the cell containing your training loop to display live results.


W&B Run: https://app.wandb.ai/awaelchli/pytorch-demo/runs/0vv7ev71

### Config
Keeping track of training parameters is easy!

In [18]:
wandb.config.batch_size = 16
wandb.config.epochs = 5

In [20]:
wandb.config.update({
    'learning_rate': 1e-3,
    'batch_norm': True,
})

print(wandb.config)

wandb_version: 1

_wandb:
  desc: null
  value:
    cli_version: 0.8.3
batch_norm:
  desc: null
  value: true
batch_size:
  desc: null
  value: 16
epochs:
  desc: null
  value: 5
learning_rate:
  desc: null
  value: 0.001



Even better: Pass your existing ```argparse``` flags to wandb!

In [3]:
import argparse
import sys
sys.argv = ['demo']
parser = argparse.ArgumentParser()
parser.add_argument('--image_height', type=int, default=128)
args = parser.parse_args()

wandb.config.update(args)  # adds all of the arguments as config variables
#print(wandb.config)

### Plotting the Error Curve

In [21]:
#%%wandb

for it in range(50):
    # ... some deep learning stuff here ...
    
    wandb.log({'loss': torch.rand(1), 'accuracy': torch.rand(1)}, step=it)


### Plotting Images

In [31]:
#%%wandb

for it in range(5):
    # ... some deep learning stuff here ...
    
    wandb.log({"examples": [wandb.Image(torch.rand(2, 3, 128, 128), caption="A caption")]})
   

### Plotting Histograms

In [8]:
#%%wandb

for it in range(5):
    # ... some deep learning stuff here ...
    
    wandb.log({"gradients": wandb.Histogram(torch.Tensor(100, 100).normal_(0, it))})


wandb: Wandb version 0.8.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


### Other 

Similarly, we can plot other data.

- Audio
- Text and Tables
- HTML
- 3D Objects (point clouds)


### Back to the Future!

Now, let's improve our [MNIST](./02_MNIST_Logging.ipynb) example.

### Summary

- Use powerful plotting tools
    - little extra effort
    - big impact
- Helpful for debugging (more later)
- Alternatives:
    - Tensorboard 
    - MLFlow
    - Visdom

<img src="./figures/tensorboard.png" width="80%"/>

<img src="./figures/visdom.png" width="80%"/>